In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import numpy as np
import pylab as pl
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier,
                              AdaBoostClassifier)
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

/home/vijay/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
from scipy.io import wavfile
import IPython.display as ipd

In [3]:
#seperate cats from dogs
train_test = pd.read_csv('DATASET/audio-cats-and-dogs/train_test_split.csv', index_col=0)

train_dog = train_test["train_dog"]
train_dog = train_dog.dropna()
train_cat = train_test["train_cat"]
train_cat = train_cat.dropna()

#print(train_cat)

dog_files =[]
cat_files =[]
for dog in train_dog:
    w = wavfile.read('DATASET/audio-cats-and-dogs/cats_dogs/'+str(dog))
    dog_files.append(w[1])
for cat in train_cat:
    w = wavfile.read('DATASET/audio-cats-and-dogs/cats_dogs/'+str(cat))
    cat_files.append(w[1])

In [5]:
cat_loudness=[]
dog_loudness=[]
x_pos = np.arange(2)
for dog in dog_files:
    dog_loudness.append(sum(abs(dog)))
for cat in cat_files:
    cat_loudness.append(sum(abs(cat)))

In [6]:
df_dog = pd.DataFrame(dog_files).fillna(int(0))
df_cat = pd.DataFrame(cat_files).fillna(int(0))[:64]

In [7]:
#calculating variance
dog_vars =[]
cat_vars = []
for row in df_dog.as_matrix():
    dog_vars.append(np.var(row))
for row in df_cat.as_matrix():
    cat_vars.append(np.var(row))

In [8]:
#calculating sum
dog_sum =[]
cat_sum = []
for row in df_dog.as_matrix():
    dog_sum.append(sum(row))
for row in df_cat.as_matrix():
    cat_sum.append(sum(row))

In [9]:
pca = PCA(n_components=2)
pca_2d_dog = pca.fit_transform(df_dog)
pca_df_dog = pd.DataFrame(pca_2d_dog,columns=["pc1","pc2"])

pca = PCA(n_components=2)
pca_2d_cat = pca.fit_transform(df_cat)
pca_df_cat = pd.DataFrame(pca_2d_cat,columns=["pc1","pc2"])

In [10]:


pca_df_dog["variance"]=dog_vars
pca_df_cat["variance"]=cat_vars
pca_df_dog["sum"]=dog_sum
pca_df_cat["sum"]=cat_sum

pca_df_dog["loudness"]=dog_loudness
pca_df_cat["loudness"]= cat_loudness[:64]
pca_df_dog["label"]=np.zeros(len(pca_df_dog))
pca_df_cat["label"]=np.ones(len(pca_df_cat))

df = pd.concat([pca_df_dog,pca_df_cat])

In [11]:
#little help function to measure accuracy
def predict_acc(X_test,y_test,clf):
    predictions=[]       
    for x in X_test.as_matrix(): 
    
        predictions.append(float(clf.predict([x])))   
    
    acc = accuracy_score(y_test, predictions)
    return [acc,predictions]

In [12]:
X = df[["pc1","pc2","variance","sum","loudness"]]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12,stratify=y)

In [13]:
accs=[]
n=10
for i in range(n):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12,stratify=y)
    # using Random Forest CLassification
    classifier =ExtraTreesClassifier()

    clf = classifier
    clf = clf.fit(X_train, y_train)
    acc = predict_acc(X_test,y_test,clf)[0]
    accs.append(acc)
print("Mean accuracy of "+str(n)+" sample splits is "+str(np.mean(accs)))

Mean accuracy of 10 sample splits is 0.9125
